# <yolov8을 이용한 segmentation 데이터셋 증강>

##### : 기존 클래스 2개(주행공간, 주차공간)을 yolov8n과 yolov8m을 사용하여 다른 사물들의 segmentation을 추가하여 증강

### 1. Ai Hub에서 다운받은 bbox2, bbox3, calib 정보가 있는 데이터셋에서 Segmentation 정보만을 추출

In [ ]:
import os
import json

# JSON 파일들이 있는 폴더 경로
json_folder_path = '/Users/hong-giheon/Desktop/엘리스 트랙 - 자율주행 트랙/088.주차 공간 탐색을 위한 차량 관점 복합 데이터/01.데이터/1.Training/라벨링데이터/실외/중소형주차장'
output_folder_path = '/Users/hong-giheon/Desktop/엘리스 트랙 - 자율주행 트랙/project_1/실외_중소형주차장_segmentation_dataset_새로운거'

# 새로운 JSON 파일로 저장하는 함수 및 크기 비교
def save_segmentation_only_json(json_folder_path, output_folder_path):
    # 출력 폴더가 없으면 생성
    os.makedirs(output_folder_path, exist_ok=True)

    original_total_size = 0
    reduced_total_size = 0

    for root, dirs, files in os.walk(json_folder_path):
        for file in files:
            if file.endswith('.json'):
                file_path = os.path.join(root, file)
                original_total_size += os.path.getsize(file_path)  # 원본 파일 크기 합산

                with open(file_path, 'r') as f:
                    data = json.load(f)

                    # segmentation이 있는 경우에만 파일 생성
                    if "segmentation" in data and data["segmentation"]:
                        # 불필요한 항목들 제거
                        data.pop("bbox2d", None)
                        data.pop("calib", None)
                        data.pop("bbox3d", None)

                        # 새로운 JSON 파일로 저장
                        relative_path = os.path.relpath(root, json_folder_path)
                        output_dir = os.path.join(output_folder_path, relative_path)
                        os.makedirs(output_dir, exist_ok=True)
                        output_file_path = os.path.join(output_dir, file)
                        with open(output_file_path, 'w') as output_file:
                            json.dump(data, output_file, indent=4)
                        
                        print(f'파일 생성됨: {output_file_path}')
                        reduced_total_size += os.path.getsize(output_file_path)  # 변환된 파일 크기 합산

    return original_total_size, reduced_total_size

# 함수 호출
original_size, reduced_size = save_segmentation_only_json(json_folder_path, output_folder_path)

# 크기 비교 출력
reduction_percentage = 100 * (original_size - reduced_size) / original_size if original_size > 0 else 0
print(f"원본 데이터셋 크기: {original_size / (1024 * 1024):.2f} MB")
print(f"축소된 데이터셋 크기: {reduced_size / (1024 * 1024):.2f} MB")
print(f"데이터셋 축소 비율: {reduction_percentage:.2f}%")


### 2. segmentation class Yolov8 이용해서 증강. 다른 사물들 세그멘테이션 후 json파일에 추가하기

In [ ]:

import os
import cv2
import json
from ultralytics import YOLO
import numpy as np

# YOLOv8 세그멘테이션 모델 로드 (사전 학습된 COCO 데이터셋 사용)
# yolov8n-seg & yolov8m-seg ...
# n -> s -> m -> l -> x 순으로 무겁고 성능 좋음
model = YOLO('yolov8n-seg.pt') 

# JSON 파일이 저장된 최상위 폴더 경로
json_base_folder = '증강시킬 데이터셋 json파일 경로'

# 이미지 파일이 저장된 최상위 폴더 경로
image_base_folder = '증강시킬 데이터셋에 맞는 이미지 파일 경로'

# 수정된 JSON 파일을 저장할 최상위 폴더 경로
output_base_folder = '증강된 데이터셋 json파일 저장 경로'

# 모든 하위 폴더를 순회하며 JSON 파일과 대응되는 이미지 파일을 처리
for root, dirs, files in os.walk(json_base_folder):
    for file in files:
        if file.endswith('.json'):
            json_file_path = os.path.join(root, file)

            # JSON 파일의 이름을 기반으로 이미지 파일 경로 구성
            relative_path = os.path.relpath(root, json_base_folder)
            image_file_name = file.replace('.json', '.jpg')
            image_file_path = os.path.join(image_base_folder, relative_path, image_file_name)

            # JSON 파일 읽기
            with open(json_file_path, 'r') as f:
                data = json.load(f)

            # 이미지 파일 읽기
            if os.path.exists(image_file_path):
                image = cv2.imread(image_file_path)

                # YOLOv8을 사용해 객체 탐지 및 세그멘테이션
                results = model(image)

                # 탐지된 객체 정보를 JSON 파일에 추가
                for result in results:
                    if result.masks is not None:  # masks가 None인지 확인
                        for i, mask in enumerate(result.masks.xy):
                            class_id = int(result.boxes.cls[i])
                            score = float(result.boxes.conf[i])

                            # 객체의 폴리곤 좌표를 얻어 JSON 포맷으로 변환
                            segmentation_info = {
                                "name": model.names[class_id],
                                "polygon": mask.tolist(),
                                "confidence": score
                            }

                            # JSON 데이터에 새로운 세그멘테이션 정보 추가
                            data["segmentation"].append(segmentation_info)
                    else:
                        print(f"No masks found for image: {image_file_path}")

                # 수정된 JSON 파일을 저장할 경로 설정
                output_folder = os.path.join(output_base_folder, relative_path)

                # 저장할 폴더가 존재하지 않으면 생성
                if not os.path.exists(output_folder):
                    os.makedirs(output_folder)

                output_json_file_path = os.path.join(output_folder, file)
                
                # 수정된 JSON 파일 저장
                with open(output_json_file_path, 'w') as f:
                    json.dump(data, f, indent=4)

                print(f"Updated JSON file saved at: {output_json_file_path}")
            else:
                print(f"Image file not found for {json_file_path}")

### 3. bounding box(bbox2) + segmentation 클래스 2개 정보가 담긴 json파일을 입력받아 yolov8을 이용해서 새로운 객체 segmentation하기. : bounding box 부분만 yolov8로 segmentation하기

In [ ]:
import os
import cv2
import json
from ultralytics import YOLO
import numpy as np

# YOLOv8 세그멘테이션 모델 로드 (사전 학습된 COCO 데이터셋 사용)
model = YOLO('yolov8m-seg.pt')

# JSON 파일이 저장된 최상위 폴더 경로
json_base_folder = '/Users/hong-giheon/Desktop/엘리스 트랙 - 자율주행 트랙/project_1/yolov8/Yolov8_bbox_seg_test_dataset/실내_대형주차장_001/label'

# 이미지 파일이 저장된 최상위 폴더 경로
image_base_folder = '/Users/hong-giheon/Desktop/엘리스 트랙 - 자율주행 트랙/project_1/yolov8/원본_image_dataset/실내_대형주차장_001/Camera'

# 수정된 JSON 파일을 저장할 최상위 폴더 경로
output_base_folder = '/Users/hong-giheon/Desktop/엘리스 트랙 - 자율주행 트랙/project_1/yolov8/Yolov8_bbox_seg_test_dataset/result_json/실내_대형주차장_001'

# bbox2d 클래스와 세그멘테이션 클래스 간의 매칭 딕셔너리
valid_classes = {
    "Car": "car",
    "Bicycle": "bicycle",
    "Motorbike": "motorcycle",
    "Van": "truck",
    "Parking Meter": "parking meter",
    "Adult": "person",  # bbox2d에서 Adult이 YOLO의 person에 해당
    "Traffic Light": "traffic light"
}

# 모든 하위 폴더를 순회하며 JSON 파일과 대응되는 이미지 파일을 처리
for root, dirs, files in os.walk(json_base_folder):
    for file in files:
        if file.endswith('.json'):
            json_file_path = os.path.join(root, file)

            # JSON 파일의 이름을 기반으로 이미지 파일 경로 구성
            relative_path = os.path.relpath(root, json_base_folder)
            image_file_name = file.replace('.json', '.jpg')
            image_file_path = os.path.join(image_base_folder, relative_path, image_file_name)

            # JSON 파일 읽기
            with open(json_file_path, 'r') as f:
                data = json.load(f)

            # 이미지 파일 읽기
            if os.path.exists(image_file_path):
                image = cv2.imread(image_file_path)

                # 기존 segmentation 정보를 유지
                combined_segmentation = data.get("segmentation", [])

                # bbox2d에서 지정한 객체들만을 위한 마스크 생성
                for bbox_info in data['bbox2d']:
                    class_name = bbox_info['name']
                    if class_name in valid_classes:
                        # bbox 좌표 정보 추출
                        x_min, y_min, x_max, y_max = bbox_info['bbox']
                        # 해당 영역만 잘라 YOLO에 입력
                        cropped_image = image[int(y_min):int(y_max), int(x_min):int(x_max)]
                        
                        # YOLOv8을 사용해 해당 영역에 대해 객체 탐지 및 세그멘테이션
                        results = model(cropped_image)

                        for result in results:
                            if result.masks is not None:
                                for i, mask in enumerate(result.masks.xy):
                                    yolo_class_id = int(result.boxes.cls[i])
                                    yolo_class_name = model.names[yolo_class_id]

                                    # bbox2d와 일치하는 클래스만 추가
                                    if yolo_class_name == valid_classes[class_name]:
                                        # 원래 이미지 좌표로 마스크 좌표 조정
                                        adjusted_mask = [[x + x_min, y + y_min] for [x, y] in mask.tolist()]
                                        score = float(result.boxes.conf[i])

                                        # 새로운 세그멘테이션 정보 추가
                                        segmentation_info = {
                                            "name": yolo_class_name,
                                            "polygon": adjusted_mask,
                                            "confidence": score
                                        }
                                        combined_segmentation.append(segmentation_info)

                # 새로운 JSON 구조 생성 (bbox2, bbox3, calib 제외)
                output_data = {
                    "image": data["image"],
                    "point_cloud": data.get("point_cloud", ""),  # point_cloud이 있을 경우 포함
                    "meta": data["meta"],
                    "segmentation": combined_segmentation
                }

                # 수정된 JSON 파일을 저장할 경로 설정
                output_folder = os.path.join(output_base_folder, relative_path)

                # 저장할 폴더가 존재하지 않으면 생성
                if not os.path.exists(output_folder):
                    os.makedirs(output_folder)

                output_json_file_path = os.path.join(output_folder, file)

                # 수정된 JSON 파일 저장
                with open(output_json_file_path, 'w') as f:
                    json.dump(output_data, f, indent=4)

                print(f"Updated JSON file saved at: {output_json_file_path}")
